In [8]:
import os
import psycopg2
from dotenv import load_dotenv
from sshtunnel import SSHTunnelForwarder
import time

In [4]:
# Load environment variables from .env file
load_dotenv("environment_variables.env")
api_key = os.getenv("API_KEY")
psql_pw = os.getenv("PSQL_PW")

In [6]:
# SSH parameters
bastion_host = 'ec2-34-224-93-62.compute-1.amazonaws.com'
bastion_user = 'ec2-user'
bastion_key = 'C:\\Users\\detto\\Documents\\ec2-key-pair.pem'

# RDS parameters
rds_host = 'youtubeviewprediction.cd0c8oow2pnr.us-east-1.rds.amazonaws.com'
rds_user = 'postgres'
rds_password = psql_pw
rds_database = 'YouTubeViewPrediction'
rds_port = 5432

In [10]:
# Create an SSH tunnel
tunnel = SSHTunnelForwarder(
    (bastion_host, 22),
    ssh_username=bastion_user,
    ssh_pkey=bastion_key,
    remote_bind_address=(rds_host, rds_port),
    local_bind_address=('localhost', 6543)  # Choose a local port for the tunnel
)

# Start the tunnel
tunnel.start()

# Connect to PostgreSQL through the tunnel
conn = psycopg2.connect(
    database=rds_database,
    user=rds_user,
    password=rds_password,
    host=tunnel.local_bind_host,
    port=tunnel.local_bind_port
)

# Create a cursor object using the cursor() method
cursor = conn.cursor()

# Execute a SQL query
cursor.execute("SELECT version();")

# Fetch result
record = cursor.fetchone()
print("You are connected to - ", record, "\n")

SSH tunnel established. Local bind address: ('127.0.0.1', 6543)
You are connected to -  ('PostgreSQL 16.1 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 7.3.1 20180712 (Red Hat 7.3.1-12), 64-bit',) 



In [ ]:

# Create an SSH client
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(bastion_host, username=bastion_user, key_filename=bastion_key)

# Start the tunnel
transport = ssh.get_transport()
local_bind_address = ('localhost', 6543)
remote_bind_address = (rds_host, rds_port)
channel = transport.open_channel("direct-tcpip", remote_bind_address, local_bind_address)

# Wait a bit for the tunnel to be established
time.sleep(1)

# Connect to PostgreSQL through the tunnel
try:
    conn = psycopg2.connect(
        database=rds_database,
        user=rds_user,
        password=rds_password,
        host=local_bind_address[0],
        port=local_bind_address[1]
    )
    print("Connected to PostgreSQL database")

    # Create a cursor object and execute a query
    cur = conn.cursor()
    cur.execute("SELECT version();")
    # Fetch result
    record = cur.fetchone()
    print("You are connected to - ", record, "\n")
except (Exception, psycopg2.Error) as error:
        print("Error while connecting to PostgreSQL", error)
    

In [ ]:
ssh -i C:\\Users\\detto\\Documents\\ec2-key-pair.pem -L 6543:youtubeviewprediction.cd0c8oow2pnr.us-east-1.rds.amazonaws.com:5432 ec2-user@34.224.93.62
